<a href="https://colab.research.google.com/github/RoboTuan/ML4IOT_HMW/blob/main/HMW2/HW2_ex2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
 pip install -q tensorflow-model-optimization

     |████████████████████████████████| 174kB 6.1MB/s 


In [2]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt
import os
from zipfile import ZipFile
import tensorflow_model_optimization as tfmot
from tensorflow_model_optimization.python.core.api.sparsity import keras as sparsity
import zlib



In [3]:
if not os.path.isdir('./HMW'):
  !git clone "https://github.com/RoboTuan/ML4IOT_HMW.git"  
  !mv 'ML4IOT_HMW' 'HMW'

ROOT_DIR = 'HMW/HMW2/'
tflite_models=ROOT_DIR + "tflite_models/"

Cloning into 'ML4IOT_HMW'...
remote: Enumerating objects: 170, done.
remote: Counting objects: 100% (170/170), done.
remote: Compressing objects: 100% (145/145), done.
remote: Total 170 (delta 93), reused 62 (delta 19), pack-reused 0
Receiving objects: 100% (170/170), 1.30 MiB | 17.05 MiB/s, done.
Resolving deltas: 100% (93/93), done.


In [4]:
#parser = argparse.ArgumentParser()
#parser.add_argument('--version', type=str, required=True, help='a,b or c')
#parser.add_argument('--mfcc', action='store_true', help='use MFCCs')
#args = parser.parse_args()

model_type='c' #args.version
mfcc=True

In [5]:
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

In [6]:
zip_path = tf.keras.utils.get_file(
        origin="http://storage.googleapis.com/download.tensorflow.org/data/mini_speech_commands.zip",
        fname='mini_speech_commands.zip',
        extract=True,
        cache_dir='.', cache_subdir='data')
data_dir = os.path.join('.', 'data', 'mini_speech_commands')
filenames = tf.io.gfile.glob(str(data_dir) + '/*/*')
filenames = tf.random.shuffle(filenames)
num_samples = len(filenames)
total = 8000

182083584/182082353 [==============================] - 1s 0us/step


In [7]:
train =  tf.convert_to_tensor(pd.read_csv('https://raw.githubusercontent.com/RoboTuan/ML4IOT_HMW/main/HMW2/kws_train_split.txt', header=None))
validation =  tf.convert_to_tensor(pd.read_csv('https://raw.githubusercontent.com/RoboTuan/ML4IOT_HMW/main/HMW2/kws_val_split.txt', header=None))
test =  tf.convert_to_tensor(pd.read_csv('https://raw.githubusercontent.com/RoboTuan/ML4IOT_HMW/main/HMW2/kws_test_split.txt', header=None))

train= tf.squeeze(train)
validation= tf.squeeze(validation)
test= tf.squeeze(test)


In [8]:
train_files = train
val_files = validation
test_files = test

In [9]:
LABELS = np.array(tf.io.gfile.listdir(str(data_dir)))
LABELS = LABELS[LABELS != 'README.md']

In [10]:
class SignalGenerator:
    def __init__(self, labels, sampling_rate, frame_length, frame_step,
            num_mel_bins=None, lower_frequency=None, upper_frequency=None,
            num_coefficients=None, mfcc=False):
        self.labels = labels
        self.sampling_rate = sampling_rate
        self.frame_length = frame_length
        self.frame_step = frame_step
        self.num_mel_bins = num_mel_bins
        self.lower_frequency = lower_frequency
        self.upper_frequency = upper_frequency
        self.num_coefficients = num_coefficients
        num_spectrogram_bins = (frame_length) // 2 + 1

        if mfcc is True:
            self.linear_to_mel_weight_matrix = tf.signal.linear_to_mel_weight_matrix(
                    self.num_mel_bins, num_spectrogram_bins, self.sampling_rate,
                    self.lower_frequency, self.upper_frequency)
            self.preprocess = self.preprocess_with_mfcc
        else:
            self.preprocess = self.preprocess_with_stft

    def read(self, file_path):
        parts = tf.strings.split(file_path, os.path.sep)
        label = parts[-2]
        label_id = tf.argmax(label == self.labels)
        audio_binary = tf.io.read_file(file_path)
        audio, _ = tf.audio.decode_wav(audio_binary)
        audio = tf.squeeze(audio, axis=1)

        return audio, label_id

    def pad(self, audio):
        zero_padding = tf.zeros([self.sampling_rate] - tf.shape(audio), dtype=tf.float32)
        audio = tf.concat([audio, zero_padding], 0)
        audio.set_shape([self.sampling_rate])

        return audio
    def get_spectrogram(self, audio):
        stft = tf.signal.stft(audio, frame_length=self.frame_length,
                frame_step=self.frame_step, fft_length=self.frame_length)
        spectrogram = tf.abs(stft)

        return spectrogram

    def get_mfccs(self, spectrogram):
        mel_spectrogram = tf.tensordot(spectrogram,
                self.linear_to_mel_weight_matrix, 1)
        log_mel_spectrogram = tf.math.log(mel_spectrogram + 1.e-6)
        mfccs = tf.signal.mfccs_from_log_mel_spectrograms(log_mel_spectrogram)
        mfccs = mfccs[..., :self.num_coefficients]

        return mfccs

    def preprocess_with_stft(self, file_path):
        audio, label = self.read(file_path)
        audio = self.pad(audio)
        spectrogram = self.get_spectrogram(audio)
        spectrogram = tf.expand_dims(spectrogram, -1)
        spectrogram = tf.image.resize(spectrogram, [32, 32])

        return spectrogram, label

    def preprocess_with_mfcc(self, file_path):
        audio, label = self.read(file_path)
        audio = self.pad(audio)
        spectrogram = self.get_spectrogram(audio)
        mfccs = self.get_mfccs(spectrogram)
        mfccs = tf.expand_dims(mfccs, -1)

        return mfccs, label

    def make_dataset(self, files, train):
        ds = tf.data.Dataset.from_tensor_slices(files)
        ds = ds.map(self.preprocess, num_parallel_calls=4)
        ds = ds.batch(32)
        ds = ds.cache()
        if train is True:
            ds = ds.shuffle(100, reshuffle_each_iteration=True)

        return ds
    



In [11]:
STFT_OPTIONS = {'frame_length': 256, 'frame_step': 128, 'mfcc': False}
MFCC_OPTIONS = {'frame_length': 640, 'frame_step': 320, 'mfcc': True,
        'lower_frequency': 20, 'upper_frequency': 4000, 'num_mel_bins': 40,
        'num_coefficients': 10}

In [12]:
#if args.mfcc is True:
if mfcc is True:
    options = MFCC_OPTIONS
    strides = [2, 1]
else:
    options = STFT_OPTIONS
    strides = [2, 2]

In [13]:
generator = SignalGenerator(LABELS, 16000, **options)
train_ds = generator.make_dataset(train_files, True)
val_ds = generator.make_dataset(val_files, False)
test_ds = generator.make_dataset(test_files, False)
units=8

#Model

##Model A

In [14]:
#MLP Model definition
mlp_model = keras.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(units=256, activation='relu'),
    keras.layers.Dense(units=256, activation='relu'),
    keras.layers.Dense(units=256, activation='relu'),
    keras.layers.Dense(units=8) #if silence 10
])

##Model B

In [15]:
#CNN_2D Model definition

cnn_model = keras.Sequential([
    keras.layers.Conv2D(filters=128,kernel_size=[3,3],strides=strides, use_bias=False),
    keras.layers.BatchNormalization(momentum=0.1),
    keras.layers.ReLU(),
    keras.layers.Conv2D(filters=128,kernel_size=[3,3],strides=strides, use_bias=False),
    keras.layers.BatchNormalization(momentum=0.1),
    keras.layers.ReLU(),
    keras.layers.Conv2D(filters=128,kernel_size=[3,3],strides=strides, use_bias=False),
    keras.layers.BatchNormalization(momentum=0.1),
    keras.layers.ReLU(),
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(units=8) #10 if silence
])

##Model C

In [16]:
ds_cnn_model = keras.Sequential([
    keras.layers.Conv2D(filters=256,kernel_size=[3,3],strides=strides, use_bias=False),
    keras.layers.BatchNormalization(momentum=0.1),
    keras.layers.ReLU(),
    keras.layers.DepthwiseConv2D(kernel_size=[3,3],strides=[1, 1], use_bias=False),
    keras.layers.Conv2D(filters=256,kernel_size=[1,1],strides=[1,1], use_bias=False),
    keras.layers.BatchNormalization(momentum=0.1),
    keras.layers.ReLU(),
    keras.layers.DepthwiseConv2D(kernel_size=[3,3],strides=[1, 1], use_bias=False),
    keras.layers.Conv2D(filters=256,kernel_size=[1,1],strides=[1,1], use_bias=False),
    keras.layers.BatchNormalization(momentum=0.1),
    keras.layers.ReLU(),
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(units=8) #10 if silence else 8
])

##Model definition

In [17]:
if model_type=='a':
  model=mlp_model
  if (mfcc==True):
    path= './callback_val_chkp/mfcc_model_a'
  else:
    path= './callback_val_chkp/stft_model_a'

if model_type=='b':
  model=cnn_model
  if (mfcc==True):
    path= './callback_val_chkp/mfcc_model_b'
  else:
    path= './callback_val_chkp/stft_model_b'

if model_type=='c':
  model=ds_cnn_model
  if (mfcc==True):
    path= './callback_val_chkp/mfcc_model_c'
  else:
    path= './callback_val_chkp/stft_model_c'

In [18]:

cp_callback = keras.callbacks.ModelCheckpoint(
    filepath=path,
    # './callback_test_chkp/chkp_best',
    monitor='val_accuracy',
    verbose=0, 
    save_best_only=True,
    # save_best_only=True,
    save_weights_only=False,
    mode='auto',
    save_freq='epoch'
)

##Training

In [19]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) ,optimizer='adam', metrics=['accuracy'])
model.fit(train_ds, epochs=20, validation_data=val_ds,callbacks=[cp_callback])

Epoch 1/20
200/200 [==============================] - ETA: 0s - loss: 1.2274 - accuracy: 0.6161WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/tracking/tracking.py:111: Model.state_updates (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: ./callback_val_chkp/mfcc_model_c/assets
200/200 [==============================] - 120s 599ms/step - loss: 1.2274 - accuracy: 0.6161 - val_loss: 0.7595 - val_accuracy: 0.7663
Epoch 2/20
200/200 [==============================] - 28s 141ms/step - loss: 0.6110 - accuracy: 0.8338 - val_loss: 0.4725 - val_accuracy: 0.8612
Epoch 3/20
200/200 [==============================] - 28s 141ms/step - loss: 0

##Validation

In [20]:
#model evaluation on validation set
val_accuracy= model.evaluate(val_ds)

25/25 [==============================] - 1s 32ms/step - loss: 0.2543 - accuracy: 0.9287


##Testing

In [21]:
test_accuracy= model.evaluate(test_ds)

25/25 [==============================] - 18s 739ms/step - loss: 0.2549 - accuracy: 0.9175


#Pruning

In [39]:
prune_low_magnitude=tfmot.sparsity.keras.prune_low_magnitude

pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,final_sparsity=0.90,begin_step=len(train_ds)*5,end_step=len(train_ds)*15)
}


model_for_pruning = prune_low_magnitude(model, **pruning_params)


In [40]:
model_for_pruning.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [41]:
callbacks = [tfmot.sparsity.keras.UpdatePruningStep()]


In [42]:
model_for_pruning.fit(train_ds,epochs=20,validation_data=val_ds,callbacks=callbacks)

Epoch 1/20
200/200 [==============================] - 27s 136ms/step - loss: 0.1384 - accuracy: 0.9547 - val_loss: 0.3404 - val_accuracy: 0.9062
Epoch 2/20
200/200 [==============================] - 27s 135ms/step - loss: 0.0862 - accuracy: 0.9736 - val_loss: 0.3176 - val_accuracy: 0.9137
Epoch 3/20
200/200 [==============================] - 27s 135ms/step - loss: 0.0830 - accuracy: 0.9759 - val_loss: 0.3190 - val_accuracy: 0.9137
Epoch 4/20
200/200 [==============================] - 27s 134ms/step - loss: 0.0538 - accuracy: 0.9856 - val_loss: 0.2822 - val_accuracy: 0.9250
Epoch 5/20
200/200 [==============================] - 27s 134ms/step - loss: 0.0599 - accuracy: 0.9839 - val_loss: 0.3231 - val_accuracy: 0.9075
Epoch 6/20
200/200 [==============================] - 27s 135ms/step - loss: 0.0325 - accuracy: 0.9937 - val_loss: 0.2337 - val_accuracy: 0.9262
Epoch 7/20
200/200 [==============================] - 27s 135ms/step - loss: 0.0167 - accuracy: 0.9986 - val_loss: 0.2485 - val_ac

In [43]:
model_for_pruning_accuracy = model_for_pruning.evaluate(
  test_ds, verbose=0)
 
print('Pruned test accuracy:', model_for_pruning_accuracy)

Pruned test accuracy: [0.3257571756839752, 0.9087499976158142]


In [44]:
model=tfmot.sparsity.keras.strip_pruning(model_for_pruning)

##Pruning2

Epoch 1/20
200/200 [==============================] - 26s 132ms/step - loss: 0.1327 - accuracy: 0.9577 - val_loss: 0.3146 - val_accuracy: 0.9187
Epoch 2/20
200/200 [==============================] - 27s 133ms/step - loss: 0.0849 - accuracy: 0.9741 - val_loss: 0.3292 - val_accuracy: 0.9112
Epoch 3/20
200/200 [==============================] - 27s 133ms/step - loss: 0.0384 - accuracy: 0.9898 - val_loss: 0.2627 - val_accuracy: 0.9237
Epoch 4/20
200/200 [==============================] - 26s 132ms/step - loss: 0.0179 - accuracy: 0.9972 - val_loss: 0.2873 - val_accuracy: 0.9162
Epoch 5/20
200/200 [==============================] - 27s 133ms/step - loss: 0.0103 - accuracy: 0.9991 - val_loss: 0.2593 - val_accuracy: 0.9350
Epoch 6/20
200/200 [==============================] - 26s 132ms/step - loss: 0.0064 - accuracy: 0.9998 - val_loss: 0.2796 - val_accuracy: 0.9237
Epoch 7/20
200/200 [==============================] - 27s 133ms/step - loss: 0.0048 - accuracy: 0.9998 - val_loss: 0.2980 - val_ac

In [47]:
model_for_pruning2 = prune_low_magnitude(model, **pruning_params)
model_for_pruning2.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [48]:
model_for_pruning2.fit(train_ds,epochs=20,validation_data=val_ds,callbacks=callbacks)

Epoch 1/20
200/200 [==============================] - 27s 136ms/step - loss: 0.0719 - accuracy: 0.9780 - val_loss: 0.5127 - val_accuracy: 0.8800
Epoch 2/20
200/200 [==============================] - 27s 133ms/step - loss: 0.0535 - accuracy: 0.9841 - val_loss: 0.4652 - val_accuracy: 0.9050
Epoch 3/20
200/200 [==============================] - 27s 133ms/step - loss: 0.0128 - accuracy: 0.9972 - val_loss: 0.3851 - val_accuracy: 0.9212
Epoch 4/20
200/200 [==============================] - 27s 133ms/step - loss: 0.0032 - accuracy: 0.9998 - val_loss: 0.3199 - val_accuracy: 0.9237
Epoch 5/20
200/200 [==============================] - 27s 133ms/step - loss: 0.0015 - accuracy: 1.0000 - val_loss: 0.3074 - val_accuracy: 0.9325
Epoch 6/20
200/200 [==============================] - 27s 134ms/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 0.3416 - val_accuracy: 0.9175
Epoch 7/20
200/200 [==============================] - 27s 134ms/step - loss: 0.0016 - accuracy: 1.0000 - val_loss: 0.2727 - val_ac

In [49]:
model=tfmot.sparsity.keras.strip_pruning(model_for_pruning)

##tflite conversion

In [50]:
ROOT_DIR="./callback_val_chkp/"
tflite_models= ROOT_DIR + "tfliteModels/"
!mkdir ./callback_val_chkp/tfliteModels


if model_type=='a':
  if (mfcc==True):
    saved_model_dir= ROOT_DIR +"mfcc_model_a"
    tflite_model_dir= tflite_models + "model_a"
  else:
    saved_model_dir= ROOT_DIR +"stft_model_a"
    tflite_model_dir= tflite_models + "model_a"
  

if model_type=='b':
  if (mfcc==True):
    saved_model_dir= ROOT_DIR +"mfcc_model_b"
    tflite_model_dir= tflite_models + "model_b"
  else:
    saved_model_dir= ROOT_DIR +"stft_model_b"
    tflite_model_dir= tflite_models + "model_b"

if model_type=='c':
  if (mfcc==True):
    saved_model_dir= ROOT_DIR +"mfcc_model_c"
    tflite_model_dir= tflite_models + "model_c"
  else:
    saved_model_dir= ROOT_DIR +"stft_model_c"
    tflite_model_dir= tflite_models + "model_c"




converter=tf.lite.TFLiteConverter.from_keras_model(model) #from_saved_model(saved_model_dir)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
#converter.representative_dataset= representative_data_gen
tflite_model=converter.convert()

with open(tflite_model_dir, "wb") as fp:
  tflite_compressed = zlib.compress(tflite_model)
  fp.write(tflite_compressed)

mkdir: cannot create directory ‘./callback_val_chkp/tfliteModels’: File exists
INFO:tensorflow:Assets written to: /tmp/tmpgkgmbt3w/assets


INFO:tensorflow:Assets written to: /tmp/tmpgkgmbt3w/assets


#Quantization

In [ ]:
'''quantize_model = tfmot.quantization.keras.quantize_model
q_aware_model = quantize_model(model)
q_aware_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
q_aware_model.fit(train_ds, epochs=20, validation_data=val_ds,callbacks=[cp_callback])

model=q_aware_model'''

##Size and latency measurement

##Size measurement

In [53]:
os.path.getsize(tflite_model_dir)

38154

##Latency measurement

In [52]:
 !python ./HMW/HMW2/kws_inference.py --model ./callback_val_chkp/tfliteModels/model_a 

2020-12-13 23:28:45.046811: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
sh: 1: cannot create /sys/devices/system/cpu/cpufreq/policy0/scaling_governor: Directory nonexistent
2020-12-13 23:28:47.128206: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-12-13 23:28:47.138347: E tensorflow/stream_executor/cuda/cuda_driver.cc:314] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2020-12-13 23:28:47.138392: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (f5683814e6a4): /proc/driver/nvidia/version does not exist
2020-12-13 23:28:47.145610: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 2299995000 Hz
2020-12-13 23:28:47.145864: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x2156840 initialized for platform Host (